<a href="https://colab.research.google.com/github/MehrdadJannesar/CNN_AdaBoost/blob/master/Payan_name_mehrdad_Clear_Codes_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mlxtend

In [0]:
from IPython.display import clear_output

import numpy as np

import warnings

import keras
from keras.models import Model,Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Activation,Dropout,Conv1D,GlobalMaxPooling1D,GRU,Bidirectional,LSTM,SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import backend as K 
from keras.utils import to_categorical

from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.plotting import plot_decision_regions
from mlxtend.plotting import plot_confusion_matrix

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn import model_selection

Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
# set parameters:
max_features = 5000
maxlen = 500
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2
embedding_vecor_length = 32
units = 128
# fix random seed for reproducibility
np.random.seed(7)
i=0

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

17465344/17464789 [==============================] - 2s 0us/step


## Create Models:

# Bidirection_GRU

In [0]:
def fit_model_BG(optimizer='adam'):
  model_BG = Sequential()
  model_BG.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
  model_BG.add(Bidirectional(GRU(100)))
  model_BG.add(Dense(1, activation='sigmoid'))
  model_BG.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model_BG


model_BG = KerasClassifier(build_fn=fit_model_BG,
                        epochs=3, 
                        batch_size=64,
                        verbose=2)
  

model_BG_param_grid = {'optimizer':['adam']}

model_BG_grid = GridSearchCV(model_BG,cv=KFold(n_splits=2),
                    param_grid=model_BG_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')


model_BG_summary = fit_model_BG()
print(model_BG_summary.summary())


# # Final evaluation of the model
# scores = model_BG.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

# prediction_BG = model_BG.predict(X_test)

# test_uniq_Bidirection_GRU = np.argmax(X_test, axis=1)


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               79800     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 201       
Total params: 240,001
Trainable params: 240,001
Non-trainable params: 0
_________________________________________________________________
None


# Bidirection_LSTM

In [0]:
def fit_model_BL(optimizer='adam'):
  model_BL = Sequential()
  model_BL.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
  model_BL.add(Bidirectional(LSTM(100)))
  model_BL.add(Dense(1, activation='sigmoid'))
  model_BL.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model_BL

model_BL = KerasClassifier(build_fn=fit_model_BL,
                        epochs=3, 
                        batch_size=64,
                        verbose=2)
  

model_BL_param_grid = {'optimizer':['adam']}

model_BL_grid = GridSearchCV(model_BL,cv=KFold(n_splits=2),
                    param_grid=model_BL_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')


model_BL_summary = fit_model_BL()
print(model_BL_summary.summary())

# print(model_BL.summary())
# model_BL.fit(X_train, y_train, epochs=3, batch_size=64)
# # Final evaluation of the model
# scores = model_BL.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

# prediction_BL = model_BL.predict(X_test)

# test_uniq_Bidirection_LSTM = np.argmax(X_test, axis=1)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 200)               106400    
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 201       
Total params: 266,601
Trainable params: 266,601
Non-trainable params: 0
_________________________________________________________________
None


# Bidirection_RNN

In [0]:
def fit_model_BR(optimizer='adam'):
  model_BR = Sequential()
  model_BR.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
  model_BR.add(Bidirectional(SimpleRNN(100)))
  model_BR.add(Dense(1, activation='sigmoid'))
  model_BR.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model_BR

model_BR = KerasClassifier(build_fn=fit_model_BR,
                        epochs=3, 
                        batch_size=64,
                        verbose=2)
  

model_BR_param_grid = {'optimizer':['adam']}

model_BR_grid = GridSearchCV(model_BR,cv=KFold(n_splits=2),
                    param_grid=model_BR_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')


model_BR_summary = fit_model_BR()
print(model_BR_summary.summary())


# print(model_BR.summary())
# model_BR.fit(X_train, y_train, epochs=3, batch_size=64)
# # Final evaluation of the model
# scores = model_BR.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

# prediction_BR = model_BR.predict(X_test)

# test_uniq_Bidirection_RNN = np.argmax(X_test, axis=1)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 200)               26600     
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 201       
Total params: 186,801
Trainable params: 186,801
Non-trainable params: 0
_________________________________________________________________
None


# LSTM


In [0]:
def fit_model_L(optimizer='adam'):
  model_L = Sequential()
  model_L.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
  model_L.add(LSTM(100))
  model_L.add(Dense(1, activation='sigmoid'))
  model_L.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model_L

model_L = KerasClassifier(build_fn=fit_model_L,
                        epochs=3, 
                        batch_size=64,
                        verbose=2)
  
model_L_param_grid = {'optimizer':['adam']}

model_L_grid = GridSearchCV(model_L,cv=KFold(n_splits=2),
                    param_grid=model_L_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')


model_L_summary = fit_model_L()
print(model_L_summary.summary())



# print(model_L.summary())
# model_L.fit(X_train, y_train, epochs=3, batch_size=64)
# # Final evaluation of the model
# scores = model_L.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

# prediction_L = model_L.predict(X_test)

# test_uniq_LSTM = np.argmax(X_test, axis=1)


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


# SimpleRNN

In [0]:
def fit_model_SR(optimizer='adam'):
  model_SR = Sequential()
  model_SR.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
  model_SR.add(SimpleRNN(100))
  model_SR.add(Dense(1, activation='sigmoid'))
  model_SR.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model_SR


model_SR = KerasClassifier(build_fn=fit_model_SR,
                        epochs=3, 
                        batch_size=64,
                        verbose=2)
  
model_SR_param_grid = {'optimizer':['adam']}

model_SR_grid = GridSearchCV(model_SR,cv=KFold(n_splits=2),
                    param_grid=model_SR_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')


model_SR_summary = fit_model_SR()
print(model_SR_summary.summary())



# print(model_SR.summary())
# model_SR.fit(X_train, y_train, epochs=3, batch_size=64)
# # Final evaluation of the model
# scores = model_SR.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

# prediction_SR = model_SR.predict(X_test)

# test_uniq_SimpleRNN = np.argmax(X_test, axis=1)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 100)               13300     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 101       
Total params: 173,401
Trainable params: 173,401
Non-trainable params: 0
_________________________________________________________________
None


# GRU

In [0]:
def fit_model_GRU(optimizer='adam'):
  model_GRU = Sequential()
  model_GRU.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
  model_GRU.add(SimpleRNN(100))
  model_GRU.add(Dense(1, activation='sigmoid'))
  model_GRU.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model_GRU


model_GRU = KerasClassifier(build_fn=fit_model_GRU,
                        epochs=3, 
                        batch_size=64,
                        verbose=2)
  
model_GRU_param_grid = {'optimizer':['adam']}

model_GRU_grid = GridSearchCV(model_GRU,cv=KFold(n_splits=2),
                    param_grid=model_GRU_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')


model_GRU_summary = fit_model_GRU()
print(model_GRU_summary.summary())


# print(model_GRU.summary())
# model_GRU.fit(X_train, y_train, epochs=3, batch_size=64)
# # Final evaluation of the model
# scores = model_GRU.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

# prediction_GRU = model_GRU.predict(X_test)

# test_uniq_SimpleRNN = np.argmax(X_test, axis=1)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 100)               13300     
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 101       
Total params: 173,401
Trainable params: 173,401
Non-trainable params: 0
_________________________________________________________________
None


## LSTM ATTENTION

In [0]:
def fit_model_L_A(optimizer='adam'):
  input_ = keras.layers.Input(shape=[max_review_length], dtype='int32')

  # get the embedding layer
  embedded = Embedding(
          input_dim=top_words,
          output_dim=32,
          input_length=max_review_length,
          trainable=False,
          mask_zero=False
      )(input_)

  activations = LSTM(units, return_sequences=True)(embedded)

  # compute importance for each step
  attention = Dense(1, activation='tanh')(activations)
  attention = keras.layers.Flatten()(attention)
  attention = keras.layers.Activation('softmax')(attention)
  attention = keras.layers.RepeatVector(units)(attention)
  attention = keras.layers.Permute([2, 1])(attention)

  multiply_layer = keras.layers.Multiply()
  sent_representation = multiply_layer([activations, attention])
  # sent_representation = keras.layers.Multiply([activations, attention])
  sent_representation = keras.layers.Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units,))(sent_representation)

  probabilities = Dense(1, activation='sigmoid')(sent_representation)
  model_L_A = Model(input=input_, output=probabilities)
  model_L_A.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

  return model_L_A



model_L_A = KerasClassifier(build_fn=fit_model_L_A,
                        epochs=3, 
                        batch_size=64,
                        verbose=2)
  
model_L_A_param_grid = {'optimizer':['adam']}

model_L_A_grid = GridSearchCV(model_L_A,cv=KFold(n_splits=2),
                    param_grid=model_L_A_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')

model_L_A_summary = fit_model_L_A()
print(model_L_A_summary.summary())







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 500, 32)      160000      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 500, 128)     82432       embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 500, 1)       129         lstm_1[0][0]               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


# Text_CNN

In [0]:
def fit_model_text_CNN(optimizer='adam'):
  model_text_CNN = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
  model_text_CNN.add(Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen))
  model_text_CNN.add(Dropout(0.2))

  # we add a Convolution1D, which will learn filters
  # word group filters of size filter_length:
  model_text_CNN.add(Conv1D(filters,
                  kernel_size,
                  padding='SAME',
                  activation='relu',
                  strides=1))
  # we use max pooling:
  model_text_CNN.add(GlobalMaxPooling1D())

  # We add a vanilla hidden layer:
  model_text_CNN.add(Dense(hidden_dims))
  model_text_CNN.add(Dropout(0.2))
  model_text_CNN.add(Activation('relu'))

  # We project onto a single unit output layer, and squash it with a sigmoid:
  model_text_CNN.add(Dense(1))
  model_text_CNN.add(Activation('sigmoid'))

  model_text_CNN.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  return model_text_CNN


model_text_CNN = KerasClassifier(build_fn=fit_model_text_CNN,
                        epochs=1, 
                        batch_size=5,
                        verbose=2)
  

model_text_CNN_param_grid = {'optimizer':['adam']}

model_text_CNN_grid = GridSearchCV(model_text_CNN,cv=KFold(n_splits=2),
                    param_grid=model_text_CNN_param_grid,
                    return_train_score=True,
                    scoring=['accuracy'],
                    refit='accuracy')

model_text_CNN_summary = fit_model_text_CNN()
print(model_text_CNN_summary.summary())



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
____

# StackingClassifier

## Running GridSearch Models

In [0]:
model_BG_grid_results = model_BG_grid.fit(X_train, y_train)
i = 0
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)

In [0]:
model_BG_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7eed0ed7f0>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

In [0]:
model_BL_grid_results = model_BL_grid.fit(X_train, y_train)
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)

2 Is Okaye!


In [0]:
model_BL_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7e9485ca20>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

In [0]:
model_BR_grid_results = model_BR_grid.fit(X_train, y_train)
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)

3 Is Okaye!


In [0]:
model_BR_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7e946abc18>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

In [0]:
model_L_grid_results = model_L_grid.fit(X_train, y_train)
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)

4 Is Okaye!


In [0]:
model_L_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7e945f3860>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

In [0]:
model_SR_grid_results = model_SR_grid.fit(X_train, y_train)
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)

5 Is Okaye!


In [0]:
model_SR_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7e944e7470>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

In [0]:
model_GRU_grid_results = model_GRU_grid.fit(X_train, y_train)
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)

6 Is Okaye!


In [0]:
model_GRU_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7e944e2a90>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

In [0]:
model_L_A_grid_results = model_L_A_grid.fit(X_train, y_train)
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)

1 Is Okaye!


In [0]:
model_L_A_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f629f1d86d8>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

In [0]:
model_text_CNN_grid_results = model_text_CNN_grid.fit(X_train, y_train)
clear_output(wait=False)
i += 1 
print(i , "Is Okaye!")
print('='*20)
print("Done!")

2 Is Okaye!
Done!


In [0]:
model_text_CNN_grid_results

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f62971fca58>,
             iid='warn', n_jobs=None, param_grid={'optimizer': ['adam']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring=['accuracy'], verbose=0)

## Stack

In [0]:
# model_BG_grid_results_clf = model_BG_grid_results
# model_BL_grid_results_clf = model_BL_grid_results
# model_BR_grid_results_clf = model_BR_grid_results
# model_L_grid_results_clf = model_L_grid_results
# model_SR_grid_results_clf = model_SR_grid_results
# model_GRU_grid_results_clf = model_GRU_grid_results
model_L_A_grid_results_clf = model_L_A_grid_results
model_text_CNN_grid_results_clf = model_text_CNN_grid_results

# Logit will be used for stacking
lr = LogisticRegression(solver='lbfgs')
sclf = StackingCVClassifier(classifiers=[#model_BG_grid_results_clf
                                        #  ,model_BL_grid_results_clf
                                        #  ,model_BR_grid_results_clf
                                        #  ,model_L_grid_results_clf
                                        #  ,model_SR_grid_results_clf
                                        #  ,model_GRU_grid_results_clf
                                         model_L_A_grid_results_clf
                                         ,model_text_CNN_grid_results_clf], 
                            meta_classifier=lr, use_probas=True, cv=2)

# Do CV
for clf, label in zip([#model_BG_grid_results_clf
                                        #  ,model_BL_grid_results_clf
                                        #  ,model_BR_grid_results_clf
                                        #  ,model_L_grid_results_clf
                                        #  ,model_SR_grid_results_clf
                                        #  ,model_GRU_grid_results_clf
                                         model_L_A_grid_results_clf
                                         ,model_text_CNN_grid_results_clf, sclf], 
                      [#'Bidirection_GRU', 
                      #  'Bidirection_LSTM', 
                      #  'Bidirection_RNN',
                      #  'LSTM',
                      #  'Simple_RNN',
                      #  'GRU',
                       'LSTM_Attention',
                       'Text_CNN']):

    scores = model_selection.cross_val_score(clf, X_train, y_train, cv=2, scoring='roc_auc')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

# Fit on train data / predict on test data
sclf_fit = sclf.fit(X_train, y_train)
mypreds = sclf_fit.predict_proba(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/3
 - 66s - loss: 0.6933 - acc: 0.5048
Epoch 2/3
 - 64s - loss: 0.6931 - acc: 0.5059
Epoch 3/3
 - 65s - loss: 0.6929 - acc: 0.5062


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/3
 - 68s - loss: 0.6935 - acc: 0.5086
Epoch 2/3
 - 65s - loss: 0.6929 - acc: 0.5157
Epoch 3/3
 - 65s - loss: 0.6956 - acc: 0.5115


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


...

Epoch 1/3
 - 142s - loss: 0.6933 - acc: 0.5011
Epoch 2/3
 - 137s - loss: 0.6930 - acc: 0.5114
Epoch 3/3
 - 136s - loss: 0.6930 - acc: 0.5150


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/3
 - 139s - loss: 0.6932 - acc: 0.5078
Epoch 2/3
 - 132s - loss: 0.6930 - acc: 0.5050
Epoch 3/3
 - 132s - loss: 0.6931 - acc: 0.5119


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/3
 - 272s - loss: 0.6928 - acc: 0.5069
Epoch 2/3
 - 265s - loss: 0.6930 - acc: 0.5118
Epoch 3/3
 - 266s - loss: 0.6931 - acc: 0.5179
Epoch 1/1
 - 19s - loss: 0.4306 - acc: 0.7801
Epoch 1/1
 - 18s - loss: 0.4252 - acc: 0.7922
Epoch 1/1
 - 31s - loss: 0.3690 - acc: 0.8287


In [0]:
print(sclf_fit)
print(mypreds)
print(model_text_CNN_grid_results_clf)
print(model_L_A_grid_results_clf)

StackingCVClassifier(classifiers=[GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
                                               error_score='raise-deprecating',
                                               estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f629f1d86d8>,
                                               iid='warn', n_jobs=None,
                                               param_grid={'optimizer': ['adam']},
                                               pre_dispatch='2*n_jobs',
                                               refit='accuracy',
                                               return_train_score=True,
                                               scoring=['accuracy...
                     meta_classifier=LogisticRegression(C=1.0,
                                                        class_weight=None,
                                                        dual=False,
                                                 

# Notes

In [0]:
# import itertools
# import numpy as np
# import matplotlib.pyplot as plt

# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix

# conf_mat = confusion_matrix(mypreds, y_test)
# acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
# print('Overall accuracy: {} %'.format(acc*100))

In [0]:
# import numpy as np
# from sklearn import datasets
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.utils import to_categorical
# from keras.wrappers.scikit_learn import KerasClassifier

# iris = datasets.load_iris()
# X= iris.data
# #Y = to_categorical(iris.target,3)
# Y = iris.target

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=1000)

# def create_model(optimizer='rmsprop'):
#     model = Sequential()
#     model.add(Dense(8,activation='relu',input_shape = (4,)))
#     model.add(Dense(3,activation='softmax'))
#     model.compile(optimizer = optimizer,
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model


# model = KerasClassifier(build_fn=create_model,
#                         epochs=10, 
#                         batch_size=5,
#                         verbose=0)

# #results = cross_val_score(model, X_train, Y_train, scoring='precision_macro')

# param_grid = {'optimizer':('rmsprop','adam')}
# grid = GridSearchCV(model,
#                     param_grid=param_grid,
#                     return_train_score=True,
#                    scoring=['precision_macro','recall_macro','f1_macro'],
#                     refit='precision_macro')
# grid_results = grid.fit(X_train,Y_train)

In [0]:
# from vecstack import stacking
# from sklearn.linear_model import LinearRegression

# # Get your data

# # Initialize 1st level estimators
# models = [LinearRegression(),
#           grid_results]

# # Get your stacked features in a single line
# S_train, S_test = stacking(models, X_train, y_train, X_test, regression=True, verbose=2)

In [0]:
# # set parameters:
# max_features = 5000
# maxlen = 500
# batch_size = 32
# embedding_dims = 50
# filters = 250
# kernel_size = 3
# hidden_dims = 250
# epochs = 2

# print('Loading data...')
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# print(len(x_train), 'train sequences')
# print(len(x_test), 'test sequences')

# print('Pad sequences (samples x time)')
# x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
# x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
# print('x_train shape:', x_train.shape)
# print('x_test shape:', x_test.shape)

# print('Build model...')
# model = Sequential()

# # we start off with an efficient embedding layer which maps
# # our vocab indices into embedding_dims dimensions
# model.add(Embedding(max_features,
#                     embedding_dims,
#                     input_length=maxlen))
# model.add(Dropout(0.2))

# # we add a Convolution1D, which will learn filters
# # word group filters of size filter_length:
# model.add(Conv1D(filters,
#                  kernel_size,
#                  padding='valid',
#                  activation='relu',
#                  strides=1))
# # we use max pooling:
# model.add(GlobalMaxPooling1D())

# # We add a vanilla hidden layer:
# model.add(Dense(hidden_dims))
# model.add(Dropout(0.2))
# model.add(Activation('relu'))

# # We project onto a single unit output layer, and squash it with a sigmoid:
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# model.summary()

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(x_test, y_test))